In [1]:
"""
从Mask文件中输出Box/Pseudo Mask（椭圆）信息，生成COCO json文件
输出：COCO json
"""

import os
import json
from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import cv2
import tqdm

dataset = {'categories':[],'images':[],'annotations':[]}
dataset['categories'].append({'id': 1, 'name': 'nodule', 'supercategory': 'mark'})
dataset['categories'].append({'id': 0, 'name': 'outlier', 'supercategory': 'mark'})

# Label ids of TN-SCUI2020 Dataset
nodule_id = 1
outlier_id = 0
category_ids = {
    '(255, 255, 255)': nodule_id,
    '(1, 1, 1)': nodule_id,
    '(0, 0, 0)': outlier_id
}

In [2]:
def create_sub_masks(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x, y))[:3]
            newpixel = ((pixel[0] > 128) * 255, (pixel[1] > 128) * 255, (pixel[2] > 128) * 255)

            # If the pixel is not black...
            if newpixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(newpixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width, height))

                # Set the pixel value to 1 (default is 0), accounting for padding
#                 sub_masks[pixel_str].putpixel((x+1, y+1), 1)
                sub_masks[pixel_str].putpixel((x, y),1)

    return sub_masks

In [3]:
def create_sub_mask_annotation(sub_mask):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    polygons = []
    j = 0
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)

        if(poly.is_empty):
            # Go to next iteration, dont save empty values in list
            continue

        polygons.append(poly)

    return polygons

In [4]:
def absolute_file_paths(image_path, mask_path, keyword, is_sub = True):
    mask_images = []

    for root, dirs, files in os.walk(os.path.abspath(mask_path)):
        for file in files:
            if 'ground' in file:
                dstFile = file.replace('_groundtruth_(1)_','').replace('.jpg_','_')
                os.rename(mask_path + '/'+ file, mask_path + '/'+ dstFile)
    
    for root, dirs, files in os.walk(os.path.abspath(image_path)):
        for file in files:
            if 'original' in file:
                dstFile = file.replace('_original_','_').replace('.jpg_','_')
                os.rename(image_path + '/'+ file, image_path + '/'+ dstFile)
                
    for root, dirs, files in os.walk(os.path.abspath(image_path)):
        for file in files:
            if '.jpg' in file:
                if file.startswith(keyword) or not is_sub:
#                 if '-' not in file:
                    mask_images.append(os.path.join(mask_path, file))
    return mask_images

In [5]:
def create_image_annotation(file_name, width, height, image_id):
    images = {
        'file_name': file_name,
        'height': height,
        'width': width,
        'id': image_id
    }
    return images

In [6]:
# Get 'images' and 'annotations' info
def images_annotations_info(image_path, mask_path, keyword, folder_id, is_sub = True):
    # This id will be automatically increased as we go
    annotation_id = 1

    annotations = []
    images = []

    # Get absolute paths of all files in a directory
    mask_images = absolute_file_paths(image_path, mask_path, keyword, is_sub)

    length = len(mask_images)
    
    for image_id, mask_image in enumerate(mask_images, folder_id):
        file_name = image_path + '/' + os.path.basename(mask_image).split('.')[0] + ".jpg"
        if image_id % 100 is 0:
            print(str(image_id) + '/' + str(length))

#         file_name = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou/mask' + os.path.basename(mask_image).split('.')[0] + ".jpg"
#         mask_image_open = cv2.imread(mask_image)
#         mask_image_open = (mask_image_open > 128) * 255
        # image shape
        mask_image_open = Image.open(mask_image)
        w, h = mask_image_open.size

        # 'images' info
        image = create_image_annotation(os.path.basename(mask_image).split('.')[0] + ".jpg", w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open.convert('RGBA'), w, h)
        for color, sub_mask in sub_masks.items():
            category_id = category_ids[color]

            # 'annotations' info
            sub_mask =  np.array(sub_mask)
            polygons = create_sub_mask_annotation(sub_mask)

            for i in range(len(polygons)):
                peudo_mask=np.zeros(mask_image_open.size).astype('uint8')
                min_x, min_y, max_x, max_y = polygons[i].bounds
                width = max_x - min_x
                height = max_y - min_y
                bbox = (int(min_x), int(min_y), int(width), int(height))
                hcenter = int((min_y + max_y) / 2)
                lcenter = int((min_x + max_x) / 2)
                peudo_mask = cv2.ellipse(peudo_mask, (lcenter, hcenter), (int(width/2), int(height/2)), 0, 0, 360, color=(255,255,255), thickness=-1, lineType=0) #画椭圆
                peudo_polygons = create_sub_mask_annotation(peudo_mask)
                area = polygons[i].area
                segmentation = []
                try:
                    peudo_mask = cv2.ellipse(peudo_mask, (lcenter, hcenter), (int(width/2), int(height/2)), 0, 0, 360, color=(255,255,255), thickness=-1, lineType=0) #画椭圆
                    peudo_polygons = create_sub_mask_annotation(peudo_mask)
                    area = peudo_polygons[0].area
                    segmentation = np.array(peudo_polygons[0].exterior.coords).ravel().tolist()
                except:
                    print(str(image_id) + '/' + str(length) + ':ERROR#' + os.path.basename(mask_image).split('.')[0])
                else:
                    annotation = {
                        'segmentation': [list(int(_) for _ in segmentation)],
                        'area': int(area),
                        'iscrowd': int(0),
                        'image_id': int(image_id),
                        'bbox': bbox,
                        'category_id': int(category_id),
                        'id': int(annotation_id)
                    }
                    annotations.append(annotation)
                    annotation_id += 1
    return images, annotations

In [7]:
# 生成A1A2A3A4json文件
#     for keyword in ['train', 'val']:
TRAIN_PATH = '/root/workspace/TN-SCUI2020-Challenge/data/train'
ORIGIN_PATH = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug'
MASK_PATH = ORIGIN_PATH + '/mask'
# for keyword in ['val', 'test', 'train']:
for folder_id, keyword in enumerate(['A1', 'A2', 'A3', 'A4'], 0):
# for keyword in ['images']:
    print(str(keyword) + ': START')
#     IMAGE_PATH = ORIGIN_PATH + '/{}'.format(keyword)
    IMAGE_PATH = ORIGIN_PATH + '/images'
    dataset['images'], dataset['annotations'] = images_annotations_info(IMAGE_PATH, MASK_PATH, keyword, folder_id * 100000)
    with open(ORIGIN_PATH + '/annotations/pseudo_' + '{}'.format(keyword) + '.json', 'w') as outfile:
        json.dump(dataset, outfile)
    print(str(keyword) + ': END')

A1: START
0/1357
25/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_12401994-a2a0-44dc-b551-76c5c8e05d52
25/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_12401994-a2a0-44dc-b551-76c5c8e05d52
100/1357
114/1357:ERROR#A1B1C1D1E1_8THY32018071710392970_A1B1C1D1E1_8THY320180717103929702_752722b4-93d4-4247-8018-68c82c62b4b8
200/1357
246/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_5737a791-1097-43ee-af0e-8de4719ba566
246/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_5737a791-1097-43ee-af0e-8de4719ba566
279/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_ce5e917b-e5f8-4566-8106-20754fc0e586
279/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY4520180823091352710V_ce5e917b-e5f8-4566-8106-20754fc0e586
279/1357:ERROR#A1B1C1D1E1_105THY4520180823091352710_A1B1C1D1E1_105THY45201808230913527

300829/12581:ERROR#A4B1C3D3E4_101THY4320180820151937778_A4B1C3D3E4_101THY4320180820151937778T_395312c5-1a9e-44f1-bfb5-5583a2d20b75
300829/12581:ERROR#A4B1C3D3E4_101THY4320180820151937778_A4B1C3D3E4_101THY4320180820151937778T_395312c5-1a9e-44f1-bfb5-5583a2d20b75
300845/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_a8e0ced2-0aee-4643-908d-2c6aea46d24d
300845/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_a8e0ced2-0aee-4643-908d-2c6aea46d24d
300845/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_a8e0ced2-0aee-4643-908d-2c6aea46d24d
300845/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_a8e0ced2-0aee-4643-908d-2c6aea46d24d
300861/12581:ERROR#A4B1C4D4E4_82THY3520180802154709877_A4B1C4D4E4_82THY3520180802154709877V_dbbbdfbd-3f16-4c05-8bbe-996a2a2eb4f7
300900/12581
300909/12581:ERROR#A4B1C3D3E4_126THY552018083015044943_A4B1C3D3E4_126THY

302000/12581
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_646ed45b-42a4-42a9-8c2f-bc64873d32b1
302073/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103

303150/12581:ERROR#A4B1C3D1E1_24THY620180718090107371_A4B1C3D1E1_24THY620180718090107371V_71829320-a700-4a03-9bc4-824812862493
303200/12581
303219/12581:ERROR#A4B1C4D4E4_82THY3520180802154709877_A4B1C4D4E4_82THY3520180802154709877V_1633c0dd-808e-4735-a08d-f95246456731
303233/12581:ERROR#A4B1C3D3E4_135THY5820180831164222001_A4B1C3D3E4_135THY5820180831164222001V_eae73324-293c-472e-9338-abb1684fde3c
303289/12581:ERROR#A4B1C3D3E4_88THY372018081608381594_A4B1C3D3E4_88THY3720180816083815940_7e79270d-ed37-43a1-a14f-affa1395e40f
303300/12581
303400/12581
303402/12581:ERROR#A4B1C4D3E2_2THY220180717092815882_A4B1C4D3E2_2THY220180717092815882T_574f13b0-bea6-4ea3-947c-c2e0adac2356
303405/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_70a69d8d-5d89-4e3c-ae5c-d9f1a1de7bec
303405/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_70a69d8d-5d89-4e3c-ae5c-d9f1a1de7bec
303405/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1

304991/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_36257764-450f-4ca1-8035-309121d94d38
304991/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_36257764-450f-4ca1-8035-309121d94d38
305000/12581
305009/12581:ERROR#A4B1C3D3E1_76THY31YAJIAYAN20180730095823757_A4B1C3D3E1_76THY31YAJIAYAN20180730095823757V_18ec3570-21b0-40e6-b1d4-a7be03599bf8
305009/12581:ERROR#A4B1C3D3E1_76THY31YAJIAYAN20180730095823757_A4B1C3D3E1_76THY31YAJIAYAN20180730095823757V_18ec3570-21b0-40e6-b1d4-a7be03599bf8
305009/12581:ERROR#A4B1C3D3E1_76THY31YAJIAYAN20180730095823757_A4B1C3D3E1_76THY31YAJIAYAN20180730095823757V_18ec3570-21b0-40e6-b1d4-a7be03599bf8
305010/12581:ERROR#A4B1C4D3E2_2THY220180717092815882_A4B1C4D3E2_2THY220180717092815882T_ab4e6452-2dfb-4654-a93d-c3938e081326
305020/12581:ERROR#A4B1C3D3E4_126THY552018083015044943_A4B1C3D3E4_126THY5520180830150449436_c91a57cd-ebe4-46c5-aa73-a22f2c13eb5c
305020/12581:ERROR#A4B1C3D3E4_126THY

...

307290/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d0cb16a7-2504-4a1c-b26f-da2ede7f29f9
307290/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d0cb16a7-2504-4a1c-b26f-da2ede7f29f9
307290/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d0cb16a7-2504-4a1c-b26f-da2ede7f29f9
307300/12581
307356/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_7357d49b-1291-422c-8d80-faecc5be6a17
307356/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_7357d49b-1291-422c-8d80-faecc5be6a17
307356/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_7357d49b-1291-422c-8d80-faecc5be6a17
307356/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_7357d49b-1291-422c-8d80-faecc5be6a17
307356/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_7357d49b

309100/12581
309149/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d44bc652-9936-4022-b6d0-d0cd7502550f
309149/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d44bc652-9936-4022-b6d0-d0cd7502550f
309149/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d44bc652-9936-4022-b6d0-d0cd7502550f
309149/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d44bc652-9936-4022-b6d0-d0cd7502550f
309149/12581:ERROR#A4B1C3D3E4_86THY372018081608371632_A4B1C3D3E4_86THY3720180816083716327_d44bc652-9936-4022-b6d0-d0cd7502550f
309200/12581
309209/12581:ERROR#A4B1C4D4E4_20637011020181015THY15120181015100301587_A4B1C4D4E4_20637011020181015THY15120181015100301587V_9f01b468-f6c8-484d-8d1e-15e6907fdb35
309300/12581
309332/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7efb034b-60bd-4790-b589-ba24121ada06
309332/12581:ERROR#A4B1C3D2E1_103T

310518/12581:ERROR#A4B1C4D4E4_20637011020181015THY15120181015100301587_A4B1C4D4E4_20637011020181015THY15120181015100301587V_6784c81c-46ff-4444-a415-e7d88ddb220a
310520/12581:ERROR#A4B1C3D4E4_19207201120180913THY13720180913112036774_A4B1C3D4E4_19207201120180913THY13720180913112036774T_35e10749-0c3c-4b12-87d7-7cd4709818b4
310600/12581
310668/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_301a4b3d-ca83-4246-a881-36a57461d594
310668/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_301a4b3d-ca83-4246-a881-36a57461d594
310668/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_301a4b3d-ca83-4246-a881-36a57461d594
310668/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_301a4b3d-ca83-4246-a881-36a57461d594
310668/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_301a4b3d-ca83-4246-a881-36a57461d594
310668/125

311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822155414957V_7c70935a-4d40-498e-9422-955af7619436
311401/12581:ERROR#A4B1C3D2E1_103THY4420180822155414957_A4B1C3D2E1_103THY4420180822

A4: END


In [8]:
# 合并A1A2A3A4json文件

#     for keyword in ['train', 'val']:
TRAIN_PATH = '/root/workspace/TN-SCUI2020-Challenge/data/train'
ORIGIN_PATH = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug'
MASK_PATH = ORIGIN_PATH + '/mask'
total_dataset = {'categories':[],'images':[],'annotations':[]}
total_dataset['categories'].append({'id': 1, 'name': 'nodule', 'supercategory': 'mark'})
total_dataset['categories'].append({'id': 0, 'name': 'outlier', 'supercategory': 'mark'})
for keyword in ['A1', 'A2', 'A3', 'A4']:
    with open(ORIGIN_PATH + '/annotations/pseudo_' + '{}'.format(keyword) + '.json', 'r') as infile:
        djson = json.loads(infile.read())
        for entity in djson['images']:
            total_dataset['images'].append({
                'file_name': entity['file_name'], 
                'id': int(entity['id']), 
                'width': int(entity['width']), 
                'height': int(entity['height'])})
        for entity in djson['annotations']:
            if len(entity['segmentation'][0]) is not 0:
                total_dataset['annotations'].append({
                    'area': entity['area'],
                    'bbox': list(int(_) for _ in entity['bbox']),
                    'category_id': int(entity['category_id']),
                    'id': int(entity['id']),
                    'image_id': int(entity['image_id']),
                    'iscrowd': int(entity['iscrowd']),
                         # mask, 矩形是从左上角点按顺时针的四个顶点
                    'segmentation': [list(int(_) for _ in entity['segmentation'][0])]})
#         total_dataset['images'].append(djson['images'])
#         total_dataset['annotations'].append(djson['annotations'])

with open(ORIGIN_PATH + '/annotations/pseudo_' + 'images.json', 'w') as outfile:
    json.dump(total_dataset, outfile)

In [10]:
# 处理tests和validations目录
TRAIN_PATH = '/root/workspace/TN-SCUI2020-Challenge/data/train'
ORIGIN_PATH = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug'
MASK_PATH = ORIGIN_PATH + '/mask'
for keyword in ['tests', 'validations']:
# for keyword in ['images']:
    sub_dataset = {'categories':[],'images':[],'annotations':[]}
    sub_dataset['categories'].append({'id': 1, 'name': 'nodule', 'supercategory': 'mark'})
    sub_dataset['categories'].append({'id': 0, 'name': 'outlier', 'supercategory': 'mark'})
    print(str(keyword) + ': START')
    IMAGE_PATH = ORIGIN_PATH + '/{}'.format(keyword)
#     IMAGE_PATH = ORIGIN_PATH + '/images'
    sub_dataset['images'], sub_dataset['annotations'] = images_annotations_info(IMAGE_PATH, MASK_PATH, keyword, 0 * 100000, False)
    with open(ORIGIN_PATH + '/annotations/pseudo_' + '{}'.format(keyword) + '.json', 'w') as outfile:
        json.dump(sub_dataset, outfile)
    print(str(keyword) + ': END')

tests: START
0/15
tests: END
validations: START
0/15
validations: END
